___________
_Notebook créé par Lise Brisset, le 21/12/2024, dans le cadre du cours de d'Apprentissage Artificiel (enseignant : Loïc Grobol)._

Groupe du projet ML2024 DEFT09 : Patricia Augustyn, Lise Brisset et Solomiia Korol.

Ce notebook permet d'extraire les données de train et de test du corpus DEFT09 et de les enregistrer en format JSON.

Il s'agit des données recueillies et transcrites du parlement européen entre 1999 et 2004. 

___________

In [74]:
# Importation des librairies nécessaires à ce programme : 
import xml.etree.ElementTree as ET
import json

## Importation des données de train en français :

#### On récupère les données de train en parcourant le fichier xml correspondant : 

Lien doc etree : https://docs.python.org/3/library/xml.etree.elementtree.html

In [ ]:
# on importe le fichier xml des données de train : 
with open('/home/lise/Documents/TAL_M2S3/apprentissage_artificiel/projet_en_groupe/deft09/Corpus d_apprentissage/deft09_parlement_appr_fr.xml', 'r') as xml_file:
    xml_tree = ET.parse(xml_file)
    root = xml_tree.getroot()

# Listes pour stocker les résultats
liste_train = []

# On récupère tous les documents :
for doc in root.findall('doc'):
    # On récupère l'attribut "id" du doc : 
    id_doc = doc.attrib['id']
    
    # On récupère le parti politique
    parti = None
    eval_parti = doc.find('.//EVAL_PARTI/PARTI')  # Utilise XPath pour trouver le parti
    if eval_parti is not None:
        parti = eval_parti.attrib['valeur']  # Récupère la valeur de l'attribut "valeur"
    
    # On récupère le texte :
    texte = []  # Initialise une liste pour stocker les paragraphes
    texte_element = doc.find('texte')  # Trouve l'élément <texte>
    if texte_element is not None:
        for simple_p in texte_element.findall('p'):  # Récupère tous les <p>
            if simple_p.text and simple_p.text.strip():  # Vérifie que le texte n'est pas vide
                texte.append(simple_p.text.strip())  # Ajoute le texte à la liste

    # On concatène les paragraphes du texte :
    texte_concatene = ' '.join(texte)  # Concatène les paragraphes

    # Ajoute les résultats à la liste
    liste_train.append({
        'id': id_doc,
        'parti': parti,
        'texte': texte_concatene
    })

# Afficher quelques résultats pour vérifier : 
for i in range(5):
    print(f"id : {liste_train[i]['id']}, parti : {liste_train[i]['parti']}, texte : {liste_train[i]['texte']}")
    print()

#### Et on enregistre au format JSON, en faisant attention à l'encodage :

In [ ]:
# on ouvre le fichier json en écriture :
with open('/home/lise/Documents/TAL_M2S3/apprentissage_artificiel/projet_en_groupe/data_parlement/train/deft09_parlement_train.json', 'w') as json_file:
    # avec train qui englobe tout le contenu de la liste_train: 
    # ensure_ascii=False pour éviter les problèmes d'encodage
    json.dump({'train': liste_train}, json_file, ensure_ascii=False, indent=4)

print("Fichier JSON créé avec succès dans le dossier data_parlement/train/ !")

# Même chose avec les données de test : 

Cette fois-ci nous devons récupérer les références des partis politiques en fonction des ids de chaque doc du fichier xml de test.

#### On récupère déjà les référence qu'on stocke dans un dictionnaire : 

In [ ]:
# Dictionnaire pour stocker les données
donnees_dict = {}

# Récupèrons les références dans le fichier des références de test : 
with open('/home/lise/Documents/TAL_M2S3/apprentissage_artificiel/projet_en_groupe/deft09/Données de référence/deft09_parlement_ref_fr.txt', 'r') as ref_file:
    # les id et les ref sont séparés par une tabulation : 
    for ligne in ref_file:
        colonnes = ligne.split('\t')
        id_ref = colonnes[0]  # La première colonne est l'ID
        classe = colonnes[1].replace("\n", "")  # La deuxième colonne est la classe
        donnees_dict[id_ref] = classe  # Ajoute l'ID et la classe au dictionnaire

# Afficher le dictionnaire
print(donnees_dict)

#### Puis on parcourt le fichier xml des données de test :

In [ ]:
# corpus de test :
with open('/home/lise/Documents/TAL_M2S3/apprentissage_artificiel/projet_en_groupe/deft09/Corpus de test/deft09_parlement_test_fr.xml', 'r') as xml_file:
    xml_tree = ET.parse(xml_file)
    root = xml_tree.getroot()

# Listes pour stocker les résultats
liste_test = []

# On récupère tous les documents :
for doc in root.findall('doc'):
    # On récupère l'attribut "id" du doc : 
    id_doc = doc.attrib['id']

    # on récupère le parti politique en fonction de l'id :
    parti = donnees_dict[id_doc]
    
    # On récupère le texte :
    texte = []  # Initialise une liste pour stocker les paragraphes
    texte_element = doc.find('texte')  # Trouve l'élément <texte>
    if texte_element is not None:
        for simple_p in texte_element.findall('p'):  # Récupère tous les <p>
            if simple_p.text and simple_p.text.strip():  # Vérifie que le texte n'est pas vide
                texte.append(simple_p.text.strip())  # Ajoute le texte à la liste

    # On concatène les paragraphes du texte :
    texte_concatene = ' '.join(texte)  # Concatène les paragraphes

    # Ajoute les résultats à la liste
    liste_test.append({
        'id': id_doc,
        'parti': parti,
        'texte': texte_concatene
    })

# Afficher quelques résultats pour vérifier :
for i in range(5):
    print(f"id : {liste_test[i]['id']}, parti : {liste_test[i]['parti']}, texte : {liste_test[i]['texte']}")
    print()

#### Et on enregistre le tout dans un fichier JSON : 

In [ ]:
# on ouvre le fichier json en écriture :
with open('/home/lise/Documents/TAL_M2S3/apprentissage_artificiel/projet_en_groupe/data_parlement/test/deft09_parlement_test.json', 'w') as json_file:
    # avec test qui englobe tout le contenu de la liste_test:
    # ensure_ascii=False pour éviter les problèmes d'encodage
    json.dump({'test': liste_test}, json_file, ensure_ascii=False, indent=4)

print("Fichier JSON créé avec succès dans le dossier data_parlement/test/ !")